In [ ]:
import pandas as pd 
import numpy as np
import plotly.graph_objs as go
import plotly.express as px
import matplotlib.pyplot as plt
from scipy import stats
import time
import datetime

import sys
sys.path.append("../../")
from utils.data_processing import _drop_consecutive_nans, add_day_ahead_column
from utils.error_metrics import _calc_mae, _calc_mse, _calc_rmse, _calc_nrmse, _calc_mape, _calc_mase, _calc_msse, _seas_naive_fcst, _calc_metrics
from utils.clustering import mapping_tsfeatures, clustering, sum_until_threshold, mapping_energy_metrics

# 1. Data Loading

In [ ]:
### load csv - 1min

df = pd.read_csv("01_load_bus.csv")
unique_id = df["ID"].unique()
df['ID'] = df['ID'].astype(str)
df['ds'] = pd.to_datetime(df['ds'])

print("Number of unique ID: ", len(unique_id))

### Mapping
mapping = pd.read_csv("02_mapping.csv")
mapping['ID'] = mapping['ID'].astype(str)

In [ ]:
### Drop all rows with no information (y=0)

zeros = df[df["y"] == 0]
id_list = zeros['ID'].unique()
print("Number of ID with y=0: ", len(id_list))


df = df[~df['ID'].isin(id_list)]
print('Number of unique ID after 0s: ', df['ID'].nunique())

In [ ]:
### Calculate Empirical Features - using tsfeatures - takes a while

calculate_new = False

if calculate_new:
    temp = df[['ds', 'ID', 'y']]
    temp['y'] = temp['y'].astype('float32')

    features = mapping_tsfeatures(df=temp, normalise=True, freq=24, calulate_tsfeatures=True, calculate_em=True)

    features.to_csv("02_features_bus.csv", index=False)
else:
    features = pd.read_csv("02_features_bus.csv")

features['ID'] = features['ID'].astype(str)
features = features.dropna()
df_pre_clu = df.copy()

# 2. Clustering

In [ ]:
### Clustering - using an 'elbow' method to determine the number of clusters

n_cluster = 40
df_cluster = df[["ID", "ds", "y"]]

clusters = clustering(
    ID_feature=features,
    df=df_cluster,
    n_cluster=n_cluster,
    normalise=True,
    n_pca=1,
    elbow=25,
    )

# 3. Mean Magnitude

In [ ]:
### Calculate the mean magnitude per ID 

df_magnitude = df.groupby("ID").mean().reset_index()

df_magnitude['ID'] = df_magnitude['ID'].astype(str)
clusters['ID'] = clusters['ID'].astype(str)

df_magnitude = df_magnitude.rename(columns={"y": "y_mean"})
df_magnitude = pd.merge(df_magnitude, mapping[["ID", "country"]], on="ID")
df_magnitude = pd.merge(df_magnitude, clusters[["ID", "cluster"]], on="ID")

# 4. Aggregation of Bus-Nodes (Mid-Level)

In [ ]:
### Merge df with df_magntiude[cluster] on ID

df = pd.merge(df, df_magnitude[["ID", "cluster"]], on="ID")
df = pd.merge(df, mapping[["ID", "country"]], on="ID")
df_magnitude['y_mean'] = df_magnitude['y_mean'].abs()

In [ ]:
### Aggregation - min

## Number of rounds (aggregations) per country & cluster - will be interrupted once only 1 ID is left
agg_no = 60

## Choose Threshold
# Local Threshold - Quantile for each country & cluster
threshold_value = 0.85
threshold_string = f"local_threshold_q{threshold_value}"

# Cluster Threshold - Quantile for each cluster
threshold_cluster = False
if threshold_cluster:
    quantile = 0.9
    threshold_string = f"cluster_threshold_q{quantile}"
    threshold_value = df_magnitude.groupby("cluster")["y_mean"].quantile(quantile).reset_index()
    threshold_value.columns = ["cluster", "threshold"]

# country Threshold - Quantile for each country
threshold_country = True
if threshold_country:
    quantile = 0.90
    threshold_string = f"country_threshold_q{quantile}"
    threshold_value = df_magnitude.groupby("country")["y_mean"].quantile(quantile).reset_index()
    threshold_value.columns = ["country", "threshold"]

# Global Threshold
threshold_global = False
if threshold_global:
    quantile = 0.9
    threshold_string = f"global_threshold_q{quantile}"
    threshold_value = df_magnitude['y_mean'].quantile(quantile)

## Aggregation
ID_to_add, df_after, mapping = sum_until_threshold(
    df=df[['ID', 'ds', 'y', 'country', 'cluster']],
    df_ID=df_magnitude[['ID', 'y_mean', 'cluster', 'country']],
    threshold_value=threshold_value,
    threshold_global=threshold_global,
    threshold_cluster=threshold_cluster,
    threshold_country=threshold_country,
    agg_no=agg_no
    )

In [ ]:
### Evaluation

df_after_test = df_after.drop_duplicates(subset=["ID", "cluster"]).reset_index(drop=True)

print("Number of unique ID in df_after for each cluster after aggregation:\n", df_after_test["cluster"].value_counts())
print('\nTotal number of unique ID in df_after:', df_after_test['ID'].nunique())

df_after.to_csv("03_df_after.csv")
df_after = df_after.rename(columns={"cluster": "cluster_after_aggregation"})

# 5. Second Clustering

In [ ]:
### Calculate new ts features after aggregation - using tsfeatures - takes a while

calculate_new = False

if calculate_new:
    ### calculate ts & em features for df_after
    df_temp = df_after[["ID", "ds", "y"]]
    df_temp['y'] = df_temp['y'].astype('float32')
    
    df_after_features = mapping_tsfeatures(df=df_temp)

    # save df_after_features
    df_after_features.to_csv("04_features_after_aggregation.csv")
else:
    df_after_features = pd.read_csv("04_features_after_aggregation.csv", index_col=0)

In [ ]:
### 2nd Clustering

n_cluster = 10
df_cluster = df_after[["ID", "ds", "y"]]

clusters = clustering(
    ID_feature=df_after_features,
    df=df_cluster,
    n_cluster=n_cluster,
    normalise=True,
    n_pca=1,
    elbow=35,
    )

clusters['ID'] = clusters['ID'].astype(str)
df_after['ID'] = df_after['ID'].astype(str)

# merge df_after with clusters
df_after = pd.merge(df_after, clusters[["ID", "cluster"]], on="ID")

In [ ]:
### Potential Subclustering - Optional!

# Which cluster?
no = 3
# How many subclusters?
n_cluster_sub = 3
# Sharp or not?
sharp = False

filtered_rows = clusters[clusters['cluster'] == no]
filtered_rows = filtered_rows.drop(columns=["cluster"])

clusters_sub = pd.DataFrame()

## Subclustering for cluster no
print(f"SUBCLUSTERING FOR CLUSTER {no}")
clusters_sub = clustering(
    ID_feature=filtered_rows,
    df=df_cluster,
    n_cluster=n_cluster_sub,
    normalise=True,
    n_pca=1,
    elbow=25,
    )

clusters_sub = clusters_sub.rename(columns={"cluster": "cluster_sub"})
clusters_sub = clusters_sub[["ID", "cluster_sub"]].reset_index(drop=True)
name = f"cluster_sub_{no}"
clusters_sub = clusters_sub.rename(columns={"cluster_sub": name})
clusters_sub["cluster_after_sub"] = df_after["cluster"].max() + clusters_sub[name] + 1


## df_after
df_after_temp = df_after[df_after["cluster"] == no]

if sharp:
    df_after = df_after[df_after["cluster"] != no]

df_after_temp = pd.merge(df_after_temp, clusters_sub[['ID', name, 'cluster_after_sub']], on="ID")
df_after_temp = df_after_temp.drop(columns=["cluster"])
df_after_temp = df_after_temp.rename(columns={"cluster_after_sub": "cluster"})


if sharp:
    df_after = pd.concat([df_after, df_after_temp], ignore_index=True)

In [ ]:
### Evaluation
df_after_test = df_after.drop_duplicates(subset=["ID", "cluster"]).reset_index(drop=True)

print("Number of unique ID in df_after for each cluster after aggregation:\n", df_after_test["cluster"].value_counts())
print('\nTotal number of unique ID in df_after:', df_after_test['ID'].nunique())

df_after.to_csv("05_df_after_95per_subclustering.csv")
mapping.to_csv("06_mapping.csv")

# 6. Weather

In [ ]:
temperature = pd.read_csv("01_load_bus.csv")
df = df.drop(columns=["cluster_after_aggregation"])

temperature['ID'] = temperature['ID'].astype(str)
temperature['ds'] = pd.to_datetime(temperature['ds'])

### Mapping
country_mapping = pd.read_csv('network_nodes.csv')
country_mapping['ID'] = country_mapping['ID'].astype(str)

temperature = pd.merge(temperature, country_mapping[["ID", "country"]], on="ID")

In [ ]:
### Calculate the mean temeprature per country

temperature = temperature[['ds', 'country', 't2m']].groupby(["ds", "country"]).mean().reset_index()
temperature['ds'] = pd.to_datetime(temperature['ds'])
temperature['country'] = temperature['country'].astype(str)

In [ ]:
### Merge with Average Country Temperature

df_temp = df.copy()
df_temp = df_temp.rename(columns={"country": "country"})
df_temp["ds"] = pd.to_datetime(df_temp["ds"])
df_temp['country'] = df_temp['country'].astype(str)
df_temp = pd.merge(df_temp, temperature, on=["country", "ds"])
df = df_temp[['ID', 'ds', 'y', 't2m', 'country', 'cluster']]

# 7. Save Data

In [ ]:
### Save Clusters of df
for j in df["cluster"].unique():
    df_cluster = df[df["cluster"] == j]
    df_cluster.to_csv(f"0{j}_load_bus_after.csv", index=False)